# Data Cleaning for Our Dataset
### Evan Schubert
### CSC 2621-141
### Created on: 04/10/2025

**What Each Column Represents**<br>
**Transaction_ID** - arbitrary unique ID given to each transaction<br>
**User_ID** - The unique identifier for each user<br>
**Transaction Amount** - amount of money transacted<br>
**Transaction Type** - ATM withdrawal, Bill Payment, POS payment, bank transfer, online purchase<br>
**Time of Transaction** - Military time for the time the transaction occured, 0 through 23<br>
**Device Used** - Tablet, mobile, desktop, unknown device<br>
**Location** - San Francisco, New York, Chicago, Boston, Houston, Miami, Los Angeles, Seattle<br>
**Previous Fraudulent Transactions** - How many previous fraudelent transactions does this user have, 0 through 4<br>
**Age** - How long the account has been active in possibly months, 0 through 119<br>
**Number of Transactions last 24H**- The number of transactions in the last 24 hours for this user, 1 through 14<br>
**Payment Method** - Debit card, credit card, UPI, net banking, invalid method<br>
**Fraudulent** - boolean representing whether the transaction was fraudulent.<br>

In [ ]:
import pandas as pd
from scipy import stats
import numpy as np
from scipy.stats import stats, linregress, kruskal, chi2_contingency
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

fraud_df = pd.read_csv("Fraud Detection Dataset.csv")

In [ ]:
## Calculate na percantages for each column
nan_counts = 100 * fraud_df.isna().sum() / len(fraud_df)
total_nan = fraud_df.isna().any(axis=1).sum()
print(nan_counts)
print(total_nan)
print(100 * total_nan / len(fraud_df))

In [ ]:
fraud_df.dropna(inplace = True, subset = ['Device_Used', 'Location', 'Payment_Method'])

In [ ]:
fraud_df.info()

In [ ]:
fraud_df.head()

In [ ]:
print(fraud_df.columns)

In [ ]:
fraud_df = fraud_df.astype({ 'Transaction_Type': 'category', 'Device_Used': 'category', 'Location': 'category','Payment_Method': 'category'})

## Filling in missing values

In [ ]:
fraud_df["Transaction_Amount_fill_mean"] = fraud_df["Transaction_Amount"]
fraud_df["Transaction_Amount_fill_median"] = fraud_df["Transaction_Amount"]
fraud_df["Transaction_Amount_fill_mode"] = fraud_df["Transaction_Amount"]

values = {"Transaction_Amount_fill_mean": fraud_df["Transaction_Amount"].mean(), "Transaction_Amount_fill_median": fraud_df["Transaction_Amount"].median(), 
         "Transaction_Amount_fill_mode": stats.mode(fraud_df["Transaction_Amount"], nan_policy="omit").mode}
fraud_df.fillna(value = values, inplace = True)

In [ ]:
fraud_df.head(10)

### Visualization

In [ ]:
type_table = pd.pivot_table(fraud_df, index="Fraudulent", values="Transaction_ID", columns="Transaction_Type", aggfunc="count", observed=True)
type_table["ATM Withdrawal"] = type_table["ATM Withdrawal"]/(type_table["ATM Withdrawal"].sum())
type_table["Bank Transfer"] = type_table["Bank Transfer"]/(type_table["Bank Transfer"].sum())
type_table["Bill Payment"] = type_table["Bill Payment"]/(type_table["Bill Payment"].sum())
type_table["Online Purchase"] = type_table["Online Purchase"]/(type_table["Online Purchase"].sum())
type_table["POS Payment"] = type_table["POS Payment"]/(type_table["POS Payment"].sum())
sns.heatmap(type_table, annot=True)

In [ ]:
device_table = pd.pivot_table(fraud_df, index="Fraudulent", values="Transaction_ID", columns="Device_Used", aggfunc="count", observed=True)
for col in device_table.columns:
    device_table[col] = device_table[col]/device_table[col].sum()
sns.heatmap(device_table, annot=True)

In [ ]:
location_table = pd.pivot_table(fraud_df, index="Fraudulent", values="Transaction_ID", columns="Location", aggfunc="count", observed=True)
for col in location_table.columns:
    location_table[col] = location_table[col]/location_table[col].sum()
sns.heatmap(location_table, annot=True)

In [ ]:
method_table = pd.pivot_table(fraud_df, index="Fraudulent", values="Transaction_ID", columns="Payment_Method", aggfunc="count", observed=False)
#type_table["ATM Withdrawal"] = type_table["ATM Withdrawal"]/(type_table["ATM Withdrawal"].sum())
for col in method_table.columns:
    method_table[col] = method_table[col]/method_table[col].sum()
sns.heatmap(method_table, annot=True)

In [ ]:
amount_plot = sns.violinplot(data=fraud_df, x="Fraudulent", y="Transaction_Amount",log_scale=True, cut=0)
amount_plot.set_title("Fraudulent Vs Transaction Amount")

### Statistical Testing

In [ ]:
from pandas.core.dtypes.common import is_numeric_dtype

#Kruskal
#Transaction_Amount, Time_of_Transaction, Previous_Fraudulent_Transactions, Account_Age, Number_of_Transactions_Last_24H
#Chai Square
#Transaction_Type, Device_Used, Location, Payment_Method
X = ["Transaction_Amount", "Time_of_Transaction", "Previous_Fraudulent_Transactions", "Account_Age", "Number_of_Transactions_Last_24H", 
    "Transaction_Type", "Device_Used", "Location", "Payment_Method"]
var_name_above=[]
test_statistic_above=[]
p_values_above=[]
for col in X:
    if is_numeric_dtype(fraud_df[col]):
        samples_by_group = []
        for value in set(fraud_df["Fraudulent"]):
            mask_nulls = ~np.isnan(fraud_df[col])
            mask = fraud_df["Fraudulent"] == value
            samples_by_group.append(fraud_df[col][mask_nulls][mask])
        stat, p = kruskal(*samples_by_group)
        var_name_above.append(col)
        test_statistic_above.append(stat)
        p_values_above.append(p)
    else:
        combination_counts = pd.crosstab(fraud_df[col], fraud_df["Fraudulent"])
        chi2, p, _, _ = chi2_contingency(combination_counts)
        var_name_above.append(col)
        test_statistic_above.append(chi2)
        p_values_above.append(p)

In [ ]:
p_values_above

## Calculate na percantages for each column

In [ ]:
nan_counts = 100*fraud_df.isna().sum()/len(fraud_df)
nan_counts

Total nan count

In [ ]:
total_nan = fraud_df.isna().any(axis=1).sum()
print(total_nan)
print(100*total_nan/len(fraud_df))

## Bootstrap generator

In [ ]:
import random
rows = []
rows.append(fraud_df[fraud_df["Fraudulent"] == 0].sample(n=int(len(fraud_df)/2),replace=True))   
rows.append(fraud_df[fraud_df["Fraudulent"] == 1].sample(n=int(len(fraud_df)/2),replace=True))
bootstrap_df = pd.concat(rows)
bootstrap_df
#todo ask if they should be balanced probabilistically like this or perfectly (exactly half and half)

In [ ]:
len(fraud_df)

## Encoding categorical values as numbers

In [ ]:
fraud_df.head()

Will dummy encode transaction_type, device_used, location, payment_method

In [ ]:
fraud_df_encoded = pd.get_dummies(fraud_df, dtype = int, columns=['Transaction_Type', 'Device_Used', 'Location', 'Payment_Method'])

In [ ]:
# DataFrame after performing dummy Encoding
pd.set_option('display.max_columns', None)
fraud_df_encoded.head()

## PCA

**This is the dataframe that contains the encoded values, with non-needed columns dropped, and rows with null values removed.**

In [ ]:
# drop non-needed columns
fraud_df_encoded.drop(inplace = True, axis=1, columns = ['Transaction_ID', 'User_ID'])

In [ ]:
# need to drop NaNs for PCA to work
fraud_df_encoded.dropna(inplace = True, axis=0)

Trying PCA on all columns, didn't work well.

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler

# scale the data (good practice so outliers don't impact as much)
scaler = StandardScaler()
data_scaled = scaler.fit_transform(fraud_df_encoded)
svd = TruncatedSVD(n_components = 2)

fraud_svd = svd.fit_transform(data_scaled)

# plot the pca
sns.scatterplot(y = fraud_svd[:,1], x = fraud_svd[:,0],  hue = fraud_df_encoded['Fraudulent']) # y is component 1, x is component 0 matrix[:,0]

Trying PCA on all original numerical columns.

In [ ]:
# NaNs should be dropped already if previous cell was ran

# scale the data (good practice so outliers don't impact as much)
scaler = StandardScaler()
data_scaled2 = scaler.fit_transform(fraud_df_encoded[['Transaction_Amount', 'Time_of_Transaction', 'Previous_Fraudulent_Transactions', 'Account_Age', 'Number_of_Transactions_Last_24H']])
svd2 = TruncatedSVD(n_components = 2)

fraud_svd2 = svd2.fit_transform(data_scaled2)

# plot the pca
sns.scatterplot(y = fraud_svd2[:,1], x = fraud_svd2[:,0],  hue = fraud_df_encoded['Fraudulent']) # y is component 1, x is component 0 matrix[:,0]

### Data Modeling and prediction

In [ ]:
#train-test split
train, test = train_test_split(fraud_df_encoded, test_size = 0.30, stratify = fraud_df_encoded["Fraudulent"])
y_train = train["Fraudulent"]
x_train = train
del x_train["Fraudulent"]
y_test = test["Fraudulent"]
x_test = test
del x_test["Fraudulent"]

In [ ]:
#Logistic Regressor
lr_model = LogisticRegression().fit(x_train, y_train)
y_pred = lr_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
#recall = recall_score(y_test, y_pred)
print("Logistic Regressor\n----------------------\nAccuracy: ", accuracy)

In [ ]:
#Decision tree
dt_model = DecisionTreeClassifier(random_state=0).fit(x_train, y_train)
y_pred = dt_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("Decision Tree\n----------------------\nAccuracy: ", accuracy)

In [ ]:
#random forest
rf_model = RandomForestClassifier().fit(x_train, y_train)
y_pred = rf_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("Random Forest\n----------------------\nAccuracy: ", accuracy)